In [47]:
import visualize;
import os;
import util;
import numpy as np;
import shutil;
import random;
import cv2;
dir_server='/home/SSD3/maheen-data';
click_str='http://vision1.idav.ucdavis.edu:1000';

def main():
    out_dir='/home/SSD3/maheen-data/horse_data_cleaned';
    new_horse_train_list=os.path.join(out_dir,'valImageList.txt');
    out_dir_im=os.path.join(dir_server,'temp');
    out_file=os.path.join(out_dir_im,'temp.png')
    data=util.readLinesFromFile(new_horse_train_list);
#     random.shuffle(data);
    img_line=data[0];
    img_line=img_line.split(' ');
    img_path=img_line[0].replace('./',out_dir+'/');
    print img_path;
    box_data=[int(num) for num in img_line[1:1+4]];
    im=cv2.imread(img_path);
    print im.shape
    cv2.rectangle(im,(box_data[0],box_data[2]),(box_data[1],box_data[3]),(255,0,0));

    points=[int(num) for num in img_line[5:]];

    points=np.array(points);

    points=np.reshape(points,(5,3));
    print points.shape
    
    colors=[(255,0,0),(0,255,0),(0,0,255),(255,255,0),(0,255,255)]
    for idx,point in enumerate(points):
        if point[2]>0:
            print point
            cv2.circle(im,(point[0],point[1]),2,colors[idx],-1);
#             break;
    cv2.imwrite(out_file,im);
    print out_file.replace(dir_server,click_str);

    
    
    
if __name__=='__main__':
    main();

/home/SSD3/maheen-data/horse_data_cleaned/_04_Aug16_png/gxy/horse+head262.png
(1000, 1000, 3)
(5, 3)
[288 348   1]
[469 373   1]
[252 716   1]
[350 759   1]
http://vision1.idav.ucdavis.edu:1000/temp/temp.png


In [ ]:
def moveData():
    out_dir='/home/SSD3/maheen-data/horse_data_cleaned';
    old_meta_path='/home/laoreja/data/horse-images/annotation';
    
    horse_train_list=os.path.join(out_dir,'valImageList_old_paths.txt');
    new_horse_train_list=os.path.join(out_dir,'valImageList.txt');
    
    
    im_paths=[line_curr.split(' ')[0] for line_curr in util.readLinesFromFile(horse_train_list)];
    new_paths=[path_curr.replace(old_meta_path,out_dir) for path_curr in im_paths];
    for old_path,new_path in zip(im_paths,new_paths):
        new_path_dir=os.path.split(new_path)[0];
        if not os.path.exists(new_path_dir):
            os.makedirs(new_path_dir);

        if not os.path.exists(new_path):
            print old_path,new_path;
            shutil.copyfile(old_path,new_path);
            
    print 'done'
    
    data_old=util.readLinesFromFile(horse_train_list);
    data_new=[line_curr.replace(old_meta_path,'.') for line_curr in data_old];
    util.writeFile(new_horse_train_list,data_new);
    print new_horse_train_list;
        

In [ ]:

def makeCleanFiles():
    out_dir='/home/SSD3/maheen-data/horse_data_cleaned';
    util.mkdir(out_dir);

    horse_old_file='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_train_allKP_minLoss.txt';
    horse_new_file='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_train_allKP_minLoss_clean.txt';
    horse_train_list='/home/laoreja/new-deep-landmark/dataset/train/trainImageList_2.txt';
    new_horse_train_list=os.path.join(out_dir,'trainImageList_old_paths.txt');
    
    horse_old_file='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss.txt';
    horse_new_file='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss_clean_noDuplicates.txt';
    horse_train_list='/home/laoreja/new-deep-landmark/dataset/train/valImageList_2.txt';
    new_horse_train_list=os.path.join(out_dir,'valImageList_old_paths.txt');
    
    horse_old_data=np.array([line_curr.split(' ')[0] for line_curr in util.readLinesFromFile(horse_old_file)]);
    horse_new_data=np.array([line_curr.split(' ')[0] for line_curr in util.readLinesFromFile(horse_new_file)]);
    
    horse_train_data=np.array(util.readLinesFromFile(horse_train_list));
    horse_train_data_pruned=[];
    
    for line_curr in horse_train_data:
        line_curr=line_curr.split(' ')[0];
        line_curr=line_curr.split('/');
        if line_curr[-2]=='gxy':
            path_pre=line_curr[-3];
        else:
            path_pre=line_curr[-2];
        line_curr=path_pre+'/'+line_curr[-1];
        line_curr=line_curr[:line_curr.rindex('.')]+'.jpg';
        horse_train_data_pruned.append(line_curr);
    
    horse_new_data_pruned=['/'.join(line_curr.split('/')[-2:]) for line_curr in horse_new_data];
    horse_problem_bin=np.in1d(horse_train_data_pruned,horse_new_data_pruned, invert=False);
    horse_train_data_new=horse_train_data[horse_problem_bin];
    print horse_train_data_new.shape;
    util.writeFile(new_horse_train_list,horse_train_data_new);
    
    
    print horse_problem_bin.shape,sum(horse_problem_bin);
    assert sum(horse_problem_bin)==len(horse_new_data_pruned);
